# 1. Install Dependencies and Setup

In [ ]:
import tensorflow as tf
import os

In [2]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

NameError: name 'tf' is not defined

In [ ]:
tf.config.list_physical_devices('GPU')

# 2. Remove dodgy images

In [1]:
import cv2
import imghdr

C:\Users\Admin\AppData\Local\Temp\ipykernel_14056\4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [ ]:
data_dir = 'data' 

In [ ]:
image_exts = ('.png', '.jpg', '.jpeg')

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            if not image_path.lower().endswith(image_exts):
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
            else:
                print('Image valid')
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            os.remove(image_path)

# 3. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data', batch_size=32, label_mode='categorical', shuffle=True)

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, axes = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    axes[idx].imshow(img.astype(int))
    axes[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

# 5. Split Data

In [ ]:
train_size = int(len(data)*.8)
val_size = int(len(data)*.1)
test_size = int(len(data)*.1)

In [ ]:
train_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 6. Build Deep Learning Model

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, SparseCategoricalAccuracy

In [ ]:
num_classes = 5
precision_metrics = [tf.keras.metrics.Precision(class_id=i) for i in range(num_classes)]
recall_metrics = [tf.keras.metrics.Recall(class_id=i) for i in range(num_classes)]

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(8, (3,3), activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1, from_logits=False),
              metrics=['accuracy'] + precision_metrics + recall_metrics)

In [ ]:
model.summary()

# 7. Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=5, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
print(hist.history)

In [ ]:
model.trainable = False
for batch in test:
    features = batch[0]
    labels = batch[1]

    # Make predictions
    predictions = model.predict(features)
    

In [ ]:
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(predictions, labels)
accuracy_value = accuracy.result()
print(accuracy_value)

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate

In [ ]:
model_precision = Precision()
model_recall = Recall()
model_accuracy = SparseCategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    model_precision.update_state(y, yhat)
    
    model_recall.update_state(y, yhat)
    model_accuracy.update_state(y, yhat)

In [ ]:
print(model_precision.result(), model_recall.result(), model_accuracy.result())

# 10. Test

In [ ]:
import cv2

In [ ]:
img = cv2.imread(os.path.join('data', 'right', 'image_1721672010.9853175.jpg'))
plt.imshow(img)
plt.show()

In [ ]:
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = tf.image.resize(img_rgb, (256,256))
plt.imshow(img_resized.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(img_resized/255, 0))

In [ ]:
yhat

In [ ]:
# if yhat > 0.5: 
#     print(f'Predicted class is Sad')
# else:
#     print(f'Predicted class is Happy')

# 11. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('model','presentation_keys.h5'))

In [ ]:
new_model = load_model(os.path.join('model','presentation_keys.h5'))

In [ ]:
new_model.predict(np.expand_dims(img_resized/255, 0))